# Background

In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need! You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this brief tutorial to understand quickly how to use it.

In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1. Obtain the 200 top-ranking universities in www.topuniversities.com

In [42]:
URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508168782318'
req = requests.get(URL)

In [45]:
import json
from pandas.io.json import json_normalize

data = json.loads(req.text)
data = json_normalize(data['data'])[:200]

df_top = pd.DataFrame(data)

df_top = df_top[['rank_display','title','country','region','score','stars']]
df_top.head()

,rank_display,title,country,region,score,stars
0,1,Massachusetts Institute of Technology (MIT),United States,North America,100,6
1,2,Stanford University,United States,North America,98.7,5
2,3,Harvard University,United States,North America,98.4,5
3,4,California Institute of Technology (Caltech),United States,North America,97.7,5
4,5,University of Cambridge,United Kingdom,Europe,95.6,5


In [98]:
df_top['total_faculties'] = 'NaN'
df_top['inter_faculties'] = 'NaN'
df_top['total_students'] = 'NaN'
df_top['inter_students'] = 'NaN'

for i in range(200):
    r =  requests.get('https://www.topuniversities.com'+data.loc[i].url)
    soup = BeautifulSoup(r.text,'html.parser')
    numbers = soup.find_all('div','number')
    
    #Handling missing data
    if(len(numbers)!=8):
        print('Missing data for '+df_top['title'].loc[i]+' in row '+str(i))
        
    else:
        df_top['total_faculties'].loc[i] = float(numbers[0].text.replace(',', ''))
        df_top['inter_faculties'].loc[i]=float(numbers[1].text.replace(',', ''))
        df_top['total_students'].loc[i]= float(numbers[2].text.replace(',', ''))
        df_top['inter_students'].loc[i] = float(numbers[3].text.replace(',', ''))





Missing data for New York University (NYU) in row 51
Missing data for Indian Institute of Science (IISc) Bangalore in row 189


In [100]:
#Handling missing data
df_top['total_faculties'].loc[189] = 423
df_top['inter_faculties'].loc[189]='Nan'
df_top['total_students'].loc[189]= 4071
df_top['inter_students'].loc[189] = 47



# Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

In [187]:
df_top_stat = df_top.copy()
df_top_stat= df_top_stat.drop(df_top_stat.index[51])
df_top_stat= df_top_stat.drop(df_top_stat.index[188])

df_top_stat['rank_display']= pd.to_numeric(df_top_stat['rank_display'].str.replace('=',''))
df_top_stat['total_faculties']= pd.to_numeric(df_top_stat['total_faculties'])
df_top_stat['inter_faculties']= pd.to_numeric(df_top_stat['inter_faculties'])
df_top_stat['total_students']= pd.to_numeric(df_top_stat['total_students'])
df_top_stat['inter_students']= pd.to_numeric(df_top_stat['inter_students'])


df_top_stat['ratio_faculty_student'] = df_top_stat.total_faculties / df_top_stat.total_students

df_top_stat['ratio_international_student'] = df_top_stat.inter_students / df_top_stat.total_students



Faculty members and students

In [188]:
df_top_stat.sort_values('ratio_faculty_student', ascending=False).head()

,rank_display,title,country,region,score,stars,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
3,4,California Institute of Technology (Caltech),United States,North America,97.7,5,953.0,350.0,2255.0,647.0,0.422616,0.286918
15,16,Yale University,United States,North America,90.4,5,4940.0,1708.0,12402.0,2469.0,0.398323,0.199081
5,6,University of Oxford,United Kingdom,Europe,95.3,5,6750.0,2964.0,19720.0,7353.0,0.342292,0.372870
4,5,University of Cambridge,United Kingdom,Europe,95.6,5,5490.0,2278.0,18770.0,6699.0,0.292488,0.356899
16,17,Johns Hopkins University,United States,North America,89.8,None,4462.0,1061.0,16146.0,4105.0,0.276353,0.254243


International students

In [189]:
df_top_stat.sort_values('ratio_international_student', ascending=False).head()

,rank_display,title,country,region,score,stars,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
34,35,London School of Economics and Political Scien...,United Kingdom,Europe,81.8,None,1088.0,687.0,9760.0,6748.0,0.111475,0.691393
11,12,Ecole Polytechnique Fédérale de Lausanne (EPFL),Switzerland,Europe,91.2,None,1695.0,1300.0,10343.0,5896.0,0.163879,0.570047
7,8,Imperial College London,United Kingdom,Europe,93.7,None,3930.0,2071.0,16090.0,8746.0,0.244251,0.543567
198,200,Maastricht University,Netherlands,Europe,47.9,None,1277.0,502.0,16385.0,8234.0,0.077937,0.502533
47,47,Carnegie Mellon University,United States,North America,78.6,None,1342.0,425.0,13356.0,6385.0,0.100479,0.478062


# Answer the previous question aggregating the data by (c) country and (d) region.

In [190]:
df_top_stat_agg = df_top_stat.copy()

Aggregate by country

In [191]:
df_top_stat_country = df_top_stat_agg[['country','total_faculties','inter_faculties','total_students',\
                                       'inter_students','ratio_faculty_student','ratio_international_student']]

df_top_stat_country = df_top_stat_country.groupby(by=['country']).agg(sum)

In [192]:
df_top_stat_country.sort_values('ratio_faculty_student', ascending=False).head()

,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
country,,,,,,
United States,152806.0,33102.0,1264839.0,218877.0,6.952958,8.960914
United Kingdom,79934.0,30216.0,583621.0,199426.0,3.805577,9.836617
Japan,28395.0,2221.0,186222.0,16269.0,1.402556,0.809999
Germany,33426.0,6402.0,342499.0,56455.0,1.083966,1.818300
Netherlands,20287.0,5683.0,197631.0,46044.0,1.053113,2.454562


In [193]:
df_top_stat_country.sort_values('ratio_international_student', ascending=False).head()

,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
country,,,,,,
United Kingdom,79934.0,30216.0,583621.0,199426.0,3.805577,9.836617
United States,152806.0,33102.0,1264839.0,218877.0,6.952958,8.960914
Australia,22034.0,11382.0,301994.0,106359.0,0.687659,3.121902
Netherlands,20287.0,5683.0,197631.0,46044.0,1.053113,2.454562
Switzerland,15323.0,9208.0,109112.0,32995.0,0.944104,2.196715


Aggregate by region

In [194]:
df_top_stat_region = df_top_stat_agg[['region','total_faculties','inter_faculties','total_students','inter_students',\
                                      'ratio_faculty_student','ratio_international_student']]

df_top_stat_region = df_top_stat_region.groupby(by=['region']).agg(sum)

In [195]:
df_top_stat_region.sort_values('ratio_faculty_student', ascending=False).head()

,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
region,,,,,,
Europe,218358.0,67598.0,1957251.0,449364.0,10.680289,21.887973
North America,182123.0,43836.0,1546353.0,292116.0,7.676045,10.729139
Asia,106311.0,25462.0,802932.0,110053.0,5.013657,5.019424
Oceania,25347.0,12786.0,350167.0,118798.0,0.825033,3.619843
Latin America,45382.0,5648.0,435750.0,36871.0,0.677452,0.502260


In [196]:
df_top_stat_region.sort_values('ratio_international_student', ascending=False).head()

,total_faculties,inter_faculties,total_students,inter_students,ratio_faculty_student,ratio_international_student
region,,,,,,
Europe,218358.0,67598.0,1957251.0,449364.0,10.680289,21.887973
North America,182123.0,43836.0,1546353.0,292116.0,7.676045,10.729139
Asia,106311.0,25462.0,802932.0,110053.0,5.013657,5.019424
Oceania,25347.0,12786.0,350167.0,118798.0,0.825033,3.619843
Latin America,45382.0,5648.0,435750.0,36871.0,0.677452,0.502260


# Obtain the 200 top-ranking universities in www.timeshighereducation.com (ranking 2018)

In [197]:
URL = 'https://www.timeshighereducation.com//sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

req = requests.get(URL)

In [223]:
import json
from pandas.io.json import json_normalize

data = json.loads(req.text)
data = json_normalize(data['data'])[:200]
df_times = pd.DataFrame(data)

df_times = df_times[['name', 'aliases', 'location', 'rank', 'stats_number_students', 'stats_pc_intl_students','stats_student_staff_ratio']]
df_times['rank'] = pd.to_numeric(df_times['rank'].str.replace('=',''))
df_times


,name,aliases,location,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,"20,409",38%,11.2
1,University of Cambridge,University of Cambridge,United Kingdom,2,"18,389",35%,10.9
2,California Institute of Technology,California Institute of Technology caltech,United States,3,"2,209",27%,6.5
3,Stanford University,Stanford University,United States,3,"15,845",22%,7.5
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,"11,177",34%,8.7
5,Harvard University,Harvard University,United States,6,"20,326",26%,8.9
6,Princeton University,Princeton University,United States,7,"7,955",24%,8.3
7,Imperial College London,Imperial College London,United Kingdom,8,"15,857",55%,11.4
8,University of Chicago,University of Chicago,United States,9,"13,525",25%,6.2
9,ETH Zurich – Swiss Federal Institute of Techno...,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,10,"19,233",38%,14.6


In [224]:
df_times['nb_students'] = df_times['stats_number_students'].str.replace(',','').astype(int)
df_times['ratio_inter_students'] = df_times['stats_pc_intl_students'].str.replace('%','').astype(float)/100
df_times['staff_student_ratio'] = 1/df_times['stats_student_staff_ratio'].astype(float)

In [225]:
df_times.head()

,name,aliases,location,rank,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,nb_students,ratio_inter_students,staff_student_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,"20,409",38%,11.2,20409,0.38,0.089286
1,University of Cambridge,University of Cambridge,United Kingdom,2,"18,389",35%,10.9,18389,0.35,0.091743
2,California Institute of Technology,California Institute of Technology caltech,United States,3,"2,209",27%,6.5,2209,0.27,0.153846
3,Stanford University,Stanford University,United States,3,"15,845",22%,7.5,15845,0.22,0.133333
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,"11,177",34%,8.7,11177,0.34,0.114943


number of faculty members

In [226]:
import numpy as np
df_times['nb_faculty_members'] =np.round((df_times['nb_students'] * df_times['staff_student_ratio'])).astype(int)
df_times['nb_inter_students'] =np.round((df_times['nb_students'] * df_times['ratio_inter_students'])).astype(int)

df_times = df_times[['rank','name','location','nb_faculty_members','nb_students','nb_inter_students','staff_student_ratio','ratio_inter_students']]
df_times.head()

,rank,name,location,nb_faculty_members,nb_students,nb_inter_students,staff_student_ratio,ratio_inter_students
0,1,University of Oxford,United Kingdom,1822,20409,7755,0.089286,0.38
1,2,University of Cambridge,United Kingdom,1687,18389,6436,0.091743,0.35
2,3,California Institute of Technology,United States,340,2209,596,0.153846,0.27
3,3,Stanford University,United States,2113,15845,3486,0.133333,0.22
4,5,Massachusetts Institute of Technology,United States,1285,11177,3800,0.114943,0.34


# Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?


Faculty members and students

In [227]:
df_times.sort_values('staff_student_ratio', ascending=False).head()

,rank,name,location,nb_faculty_members,nb_students,nb_inter_students,staff_student_ratio,ratio_inter_students
105,105,Vanderbilt University,United States,3640,12011,1561,0.303030,0.13
109,109,University of Copenhagen,Denmark,7413,30395,4255,0.243902,0.14
153,153,University of Rochester,United States,2241,9636,2794,0.232558,0.29
11,12,Yale University,United States,2827,12155,2553,0.232558,0.21
12,13,Johns Hopkins University,United States,3604,15498,3720,0.232558,0.24


International students

In [228]:
df_times.sort_values('ratio_inter_students', ascending=False).head()

,rank,name,location,nb_faculty_members,nb_students,nb_inter_students,staff_student_ratio,ratio_inter_students
24,25,London School of Economics and Political Science,United Kingdom,825,10065,7146,0.081967,0.71
178,179,University of Luxembourg,Luxembourg,340,4969,2832,0.068493,0.57
37,38,École Polytechnique Fédérale de Lausanne,Switzerland,886,9928,5460,0.089286,0.55
7,8,Imperial College London,United Kingdom,1391,15857,8721,0.087719,0.55
102,103,Maastricht University,Netherlands,929,16727,8364,0.055556,0.50


# Answer the previous question aggregating the data by (c) country and (d) region.

In [229]:
df_times_agg= df_times.copy()

In [230]:
df_times_country = df_times_agg[['location','nb_faculty_members','nb_students','nb_inter_students',\
                                 'staff_student_ratio','ratio_inter_students']].groupby(by=['location']).agg(sum)

In [231]:
df_times_country.sort_values('staff_student_ratio', ascending=False)

,nb_faculty_members,nb_students,nb_inter_students,staff_student_ratio,ratio_inter_students
location,,,,,
United States,152354,1705533,280365,6.596162,11.27
United Kingdom,44439,596449,213058,2.301821,11.33
Netherlands,13606,240180,41565,0.734555,2.50
China,21390,253911,22883,0.611938,0.60
France,7437,99378,18975,0.596114,1.25
Switzerland,10049,107852,32746,0.584128,2.20
Germany,15355,586364,87490,0.559135,3.04
Denmark,10598,64110,9327,0.480499,0.50
Sweden,8484,125746,16009,0.434852,0.84


In [232]:
df_times_country.sort_values('ratio_inter_students', ascending=False)

,nb_faculty_members,nb_students,nb_inter_students,staff_student_ratio,ratio_inter_students
location,,,,,
United Kingdom,44439,596449,213058,2.301821,11.33
United States,152354,1705533,280365,6.596162,11.27
Germany,15355,586364,87490,0.559135,3.04
Netherlands,13606,240180,41565,0.734555,2.50
Australia,9940,268630,83812,0.305868,2.46
Switzerland,10049,107852,32746,0.584128,2.20
Hong Kong,4141,77663,25158,0.283110,1.64
Canada,13240,249401,55906,0.316357,1.35
France,7437,99378,18975,0.596114,1.25


# Merge

The idea is to modify the name of each university to make them as equal as possible:
 - Lower all letters
 - Translating important words (university, school, technical..)
 - Get rid of special characters
 - Get rid of prepositions
 - Get rid of parenthesis and their content. Example: (UCB)


In [233]:
df_top_tomerge = df_top_stat.copy()
df_times_tomerge = df_times.copy()

In [234]:
def modify_tomerge(string):
    
    #lower case
    string = string.str.lower()
    
    # University
    string = string.str.replace('universite','university')
    string = string.str.replace('universitat','university')
    string = string.str.replace('universitaet','university')
    string = string.str.replace('universidad','university')
    string = string.str.replace('universiteit','university')
    string = string.str.replace('universidade','university')
    string = string.str.replace('universitari','university')


    # school
    string = string.replace('scuola','school')
    
    #technical
    string = string.replace('technische','technical')
    
    #studies
    string = string.replace('studi','studies')
    string = string.replace('estudios','studies')


    #some translations
    string = string.replace('freie','free')
    string = string.replace('tecnológico','technological')



    #special characters
    string = string.str.replace('é','e')
    string = string.str.replace('-','')
    string = string.str.replace('ä','a')
    string = string.str.replace('ã','a')
    string = string.str.replace('ó','o')
    string = string.str.replace('ö','o')
    string = string.str.replace('&','')
    string = string.str.replace('/','')


    #prepositions
    string = string.str.replace('of','')
    string = string.str.replace('the','')
    string = string.str.replace('at','')
    string = string.str.replace('de','')
    string = string.str.replace('y','')
    string = string.str.replace('di','')
    string = string.str.replace(',','')


    #parenthesis and their content
    string = string.str.replace(r"\(.*\)","")
    
    #space
    string = string.str.replace(' ','')
    

    return string
   


In [235]:
df_top_tomerge = df_top_tomerge.rename(columns={'title': 'name', 'rank_display':'rank_top',\
                                                'total_faculties':'total_fac_top','inter_faculties':'inter_fac_top',\
                                                'total_students':'total_students_top','inter_students':'inter_students_top',\
                                                 'ratio_faculty_student':'ratio_fac_students_top',\
                                                'ratio_international_student':'ratio_inter_students_top'})


df_times_tomerge = df_times_tomerge.rename(columns={'rank': 'rank_times','location':'country',\
                                                    'nb_faculty_members':'total_fac_times',\
                                                    'nb_students':'total_students_times',\
                                                    'nb_inter_students':'inter_students_times',\
                                                    'staff_student_ratio':'ratio_fac_students_times',\
                                                    'ratio_inter_students':'ratio_inter_students_times'})
df_times_tomerge['name'] = modify_tomerge(df_times_tomerge['name'])
df_top_tomerge['name'] = modify_tomerge(df_top_tomerge['name'])


df = df_top_tomerge.merge(df_times_tomerge, how='inner')
df= df[['rank_top','rank_times','name','country','region','total_fac_top','total_fac_times','total_students_top',\
       'total_students_times','inter_students_top','inter_students_times','ratio_fac_students_top',\
        'ratio_fac_students_times','ratio_inter_students_top','ratio_inter_students_times']]
df                                    

,rank_top,rank_times,name,country,region,total_fac_top,total_fac_times,total_students_top,total_students_times,inter_students_top,inter_students_times,ratio_fac_students_top,ratio_fac_students_times,ratio_inter_students_top,ratio_inter_students_times
0,1,5,massachusettsinstitutetechnolog,United States,North America,2982.0,1285,11067.0,11177,3717.0,3800,0.269450,0.114943,0.335863,0.34
1,2,3,stanforduniversit,United States,North America,4285.0,2113,15878.0,15845,3611.0,3486,0.269870,0.133333,0.227422,0.22
2,3,6,harvarduniversit,United States,North America,4350.0,2284,22429.0,20326,5266.0,5285,0.193945,0.112360,0.234785,0.26
3,4,3,californiainstitutetechnolog,United States,North America,953.0,340,2255.0,2209,647.0,596,0.422616,0.153846,0.286918,0.27
4,5,2,universitcambridge,United Kingdom,Europe,5490.0,1687,18770.0,18389,6699.0,6436,0.292488,0.091743,0.356899,0.35
5,6,1,universitoxford,United Kingdom,Europe,6750.0,1822,19720.0,20409,7353.0,7755,0.342292,0.089286,0.372870,0.38
6,8,8,imperialcollegelondon,United Kingdom,Europe,3930.0,1391,16090.0,15857,8746.0,8721,0.244251,0.087719,0.543567,0.55
7,9,9,universitchicago,United States,North America,2449.0,2181,13557.0,13525,3379.0,3381,0.180645,0.161290,0.249244,0.25
8,11,52,nanangtechnologicaluniversitsingapore,Singapore,Asia,4338.0,1564,25738.0,25499,7251.0,7905,0.168545,0.061350,0.281724,0.31
9,12,38,ecolepoltechniqueferalelausanne,Switzerland,Europe,1695.0,886,10343.0,9928,5896.0,5460,0.163879,0.089286,0.570047,0.55


Correlation within the data frames

In [239]:
df[['inter_students_top','ratio_inter_students_top']].corrwith(df['rank_top'])


inter_students_top         -0.223240
ratio_inter_students_top   -0.281845
dtype: float64

In [240]:
df[['inter_students_times','ratio_inter_students_times']].corrwith(df['rank_times'])



inter_students_times         -0.259077
ratio_inter_students_times   -0.166482
dtype: float64

In [213]:
df.corr()

,rank_top,total_fac_top,total_fac_times,total_students_top,total_students_times,inter_students_top,inter_students_times,ratio_fac_students_top,ratio_fac_students_times,ratio_inter_students_top,ratio_inter_students_times
rank_top,1.000000,-0.360142,-0.231980,-0.014808,-0.014556,-0.223240,-0.249575,-0.408615,-0.192751,-0.281845,-0.270298
total_fac_top,-0.360142,1.000000,0.737898,0.600140,0.621582,0.387949,0.359093,0.308711,0.139962,-0.142571,-0.198271
total_fac_times,-0.231980,0.737898,1.000000,0.526713,0.551056,0.156651,0.151103,0.148695,0.437160,-0.306139,-0.331997
total_students_top,-0.014808,0.600140,0.526713,1.000000,0.981961,0.631455,0.577871,-0.438447,-0.385858,-0.227782,-0.310580
total_students_times,-0.014556,0.621582,0.551056,0.981961,1.000000,0.579300,0.549183,-0.421323,-0.383636,-0.262046,-0.342533
inter_students_top,-0.223240,0.387949,0.156651,0.631455,0.579300,1.000000,0.922504,-0.281232,-0.374529,0.500943,0.376758
inter_students_times,-0.249575,0.359093,0.151103,0.577871,0.549183,0.922504,1.000000,-0.270746,-0.376410,0.511905,0.481763
ratio_fac_students_top,-0.408615,0.308711,0.148695,-0.438447,-0.421323,-0.281232,-0.270746,1.000000,0.696818,0.085412,0.113392
ratio_fac_students_times,-0.192751,0.139962,0.437160,-0.385858,-0.383636,-0.374529,-0.376410,0.696818,1.000000,-0.095414,-0.049771
ratio_inter_students_top,-0.281845,-0.142571,-0.306139,-0.227782,-0.262046,0.500943,0.511905,0.085412,-0.095414,1.000000,0.953138
